In [1]:
# import wandb
# wandb.login()
# wandb.init(project='topic-modelling')

# %%wandb
# wandb.log({'acc': random.random()}, commit=False)

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

from top2vec import Top2Vec

from matplotlib import pyplot as plt
import plotly.express as px

In [3]:
path_corpora = Path('data/corpora')
path_corpus_tut = path_corpora / Path('clean_tutby_126784.csv')

path_model_top2vec = Path('data/model/top2vec_clean_126784.joblibfile')
path_points = Path('data/model/points_clean_126784.npy')
path_plot = Path('data/plot/embeddings_clean_126784.html')


words = ['протест']  # ['насилие', 'сила', 'арест', 'задержания', 'избиения']

In [4]:
data = pd.read_csv(path_corpus_tut)
data['tags'] = data['tags'].apply(eval)

corpus = data['document']

display(data.head(3))
print(corpus.shape)

,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,главный тренер солигорский шахтер олег кубарев...,[футбол]
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,план восстановление рисунок пока художник илья...,"[архитектура, живопись, ЖКХ]"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,областной управление мчс день пожарный служба ...,[министерства]


(126784,)


In [5]:
%%time

model = Top2Vec.load(path_model_top2vec)

reduced = True
model.hierarchical_topic_reduction(num_topics=100)


n_topics = model.get_num_topics(reduced=reduced)
print('\n n_topics', n_topics)

topic_sizes, topic_nums = model.get_topic_sizes(reduced=reduced)
print('\n topic_sizes, topic_nums', topic_sizes, topic_nums)

topic_words, word_scores, topic_nums = model.get_topics(reduced=reduced)
print('\n topic_words, word_scores, topic_nums', topic_words, word_scores, topic_nums)

documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=0, num_docs=3, reduced=reduced)
print('\n search_documents_by_topic', documents, document_scores, document_ids)

topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=words, num_topics=5, reduced=reduced)
print('\n search_topics', topic_words, word_scores, topic_scores, topic_nums)

documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=words, num_docs=3)
print('\n search_documents_by_keywords', documents, document_scores, document_ids)

words, word_scores = model.similar_words(keywords=words, num_words=7)
for word, score in zip(words, word_scores):
    print(f'{word} {score}')
    
# model.add_documents(documents, document_ids=None)
# model.search_documents_by_documents(doc_ids, num_docs)


 n_topics 100

 topic_sizes, topic_nums [4356 2538 2257 2143 1942 1931 1828 1681 1640 1639 1627 1624 1589 1541
 1521 1520 1508 1505 1486 1478 1461 1459 1432 1398 1395 1393 1387 1371
 1364 1363 1355 1347 1330 1319 1317 1315 1313 1313 1304 1270 1267 1254
 1246 1225 1217 1213 1207 1202 1198 1189 1187 1187 1162 1150 1144 1127
 1126 1122 1116 1114 1111 1100 1089 1083 1054 1052 1050 1043 1041 1029
 1023 1022 1017 1010 1001 1001  996  993  988  977  974  973  963  962
  962  949  921  916  915  910  907  884  858  854  840  833  827  822
  815  806] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]

 topic_words, word_scores, topic_nums [['дтп' 'пдд' 'водитель' ... 'пьяныи' 'пристегнуть' 'кювет']
 ['бате' 'борисовчанин' 'городея' ... 'трене


 search_documents_by_keywords ['гонконгский полиция применять слезоточивый газ разгонять тысяча демонстрант собираться полуостров коулун это воскресение сообщать агентство reuters фото reuters архив протестующий собираться очередной антиправительственный митинг несмотря запрет власть многие маска несколько человек арестовывать демонстрант госпитализировать четыре месяц назад гонконг начинаться массовый акция протест против попытка власть принимать законопроект экстрадиция позволять выдавать правонарушитель материковый китай несмотря решение глава администрация кэрри лама отказываться инициатива многий сторонник оппозиция число молодежь выражать готовность продолжать свой выступление усиливаться выходной праздничный день периодически сопровождаться беспорядок демонстрант частность требовать демократизация избирательный система гонконг освобождение весь задержанный участник протестный акция'
 'очередной акция протест защита демократия гонконг воскресение выливаться столкновение полиция 

In [6]:
k = 2837

document = model.documents[k]

topic = model.doc_top[k] if not reduced else model.doc_top_reduced[k]
words = model.topic_words[topic] if not reduced else model.topic_words_reduced[topic]

documents, doc_scores, doc_ids = model.search_documents_by_documents(doc_ids=[k], num_docs=3)

print(document)
print(words)
print(documents)

управление область расследовать уголовный дело обвинение житель витебск совершение кража дачный участок подробность tutby сообщать официальный представитель ведомство регион светлана сахаров обвиняемый похищать дача инструмент стройматериал утварь дерево фото следственный комитет летний жительница областной центр летний сожитель ездить область машина похищать имущество дачный дом приусадебный участок следователь насчитывать потерпевший кража происходить январь июнь год витебский лиозненский ушачский район пара интересовать мотоблок электроинструмент радиатор водяной насос рыболовный банный принадлежность строительный материал спиртной напиток велосипед кроме сожитель забирать участок туя можжевельник растение пример дача лиозненский район женщина выкапывать туя стоимость рубль целое обвиняемый похищать имущество сумма рубль чтото украсть продавать обычный способ чтото интернет часть похищать милиция изымать дом сожитель витебск находить вернуть потерпевший мужчина женщина обвинять краж

In [7]:
%%time

n_predicted_tags = 5


with open(path_points, 'rb') as file:
    points = np.load(file)


colors = {
    'акции протеста': 'red',
    'Коронавирус': 'orange',
    'Погода': 'lightblue',
    'ДТП': 'yellow',
    'Здравоохранение': 'darkblue',
    'Автомобили': 'lightyellow',
    'футбол': 'green',
    'Гаджеты': 'pink',
    'В мире': 'salmon',
    'криминал': 'brown',
}
color_default = 'lightgray'

def tags_to_color(arr):
    for item in [colors.get(_, color_default) for _ in arr]:
        if item != color_default:
            return item
    return color_default

color = data['tags'].apply(tags_to_color)


hover_data = {
    'url': data['url'], 
    'label': data['label'], 
    'header': data['header'], 
    'tags': data['tags'], 
    'predicted_tags': model.topic_words_reduced[model.doc_top_reduced][:, :n_predicted_tags].tolist()
}

fig = px.scatter(
    x=points[:, 0], 
    y=points[:, 1],
    color=color,
    color_discrete_map='identity',
    hover_data=hover_data, 
    width=1000, 
    height=1000,
)
fig.update_traces(marker=dict(size=4))

fig.write_html(str(path_plot))

Wall time: 18.3 s
